# Importando dados

In [2]:
import pandas as pd
import math
import random

In [3]:
df = pd.read_csv("../data/cmc.data.csv")
df.head()

,24,2,3,3.1,1,1.1,2.1,3.2,0,1.2
0,45,1,3,10,1,1,3,4,0,1
1,43,2,3,7,1,1,3,4,0,1
2,42,3,2,9,1,1,3,3,0,1
3,36,3,3,8,1,1,3,2,0,1
4,19,4,4,0,1,1,3,3,0,1


## Adicionando colunas

In [4]:
colunas = ["idade", "escolaridade", "escolaridade_marido", "no_filhos", "religiao", "trabalha",
           "ocupacao_marido", "indice_padrao_vida", "exposicao_midia", "contraceptivo"]
df.columns = colunas
df.head()

,idade,escolaridade,escolaridade_marido,no_filhos,religiao,trabalha,ocupacao_marido,indice_padrao_vida,exposicao_midia,contraceptivo
0,45,1,3,10,1,1,3,4,0,1
1,43,2,3,7,1,1,3,4,0,1
2,42,3,2,9,1,1,3,3,0,1
3,36,3,3,8,1,1,3,2,0,1
4,19,4,4,0,1,1,3,3,0,1


# Funções

In [5]:
def calcular_entropia(df, coluna):
    df_aux = df[[coluna, "contraceptivo"]]
    s = list(df[coluna].unique())
    
    contagem = {}
    entropia = {}
    for item in s:
        chave = str(item)

        contagem[chave] = []
        entropia[chave] = 0

        df_item = df_aux[df_aux[coluna] == item]
        for contraceptivo in [1,2,3]:
            contagem[chave].append(len(df_item[df_item["contraceptivo"] == contraceptivo]))
        contagem[chave].append(len(df_item))

    for item in s:
        for i in [0,1,2]:
            if contagem[str(item)][-1] != 0:
                fracao = contagem[str(item)][i] / contagem[str(item)][-1]
                if fracao > 0:
                #     print(contagem)
                # print(contagem[str(item)][i], contagem[str(item)][-1], fracao)
                    entropia[str(item)] -= fracao * math.log2(fracao)

    entropia_coluna = 0
    tamanho_df = len(df)

    for item in s:
        entropia_coluna += (contagem[str(item)][-1] / tamanho_df) * entropia[str(item)]

    return es(df) - entropia_coluna

In [6]:
def treino_teste(df, porcentagem):
    index = list(range(len(df)))
    porcentagem = int(1/porcentagem)
    
    treino = random.sample(index, (len(df) // porcentagem))
    teste = [i for i in index if i not in treino]

    treino = df.loc[df.index.isin(treino)]
    teste = df.loc[df.index.isin(teste)]

    return treino, teste

In [7]:
# DEVE SER CALCULADO APENAS UMA VEZ
def es(df):
    contraceptivos = list(df["contraceptivo"].unique())
    tamanho_df = len(df)
    valor_es = 0

    for item in contraceptivos:
        fracao = len(df[df["contraceptivo"] == item])/ tamanho_df
        if fracao != 0:
            valor_es -= fracao * math.log2(fracao)
    
    return valor_es

In [8]:
# PASSAR APENAS UMA VEZ OS DADOS NUMÉRICOS PARA CATEGÓRIGOS
def categorizacao(df):
    for i in range(len(df)):
        if isinstance(df.iloc[i, 0], str):
            pass
        elif df.iloc[i, 0] <= 26:
            df.iloc[i, 0] = "<=26"
        elif df.iloc[i, 0] > 26 and df.iloc[i, 0] <= 32:
            df.iloc[i, 0] = "26...32"
        elif df.iloc[i, 0] > 32 and df.iloc[i, 0] <= 39:
            df.iloc[i, 0] = "32...39"
        elif df.iloc[i, 0] > 39:
            df.iloc[i, 0] = ">39"

        if isinstance(df.iloc[i, 3], str):
            pass
        elif df.iloc[i, 3] <= 5:
            df.iloc[i, 3] = "<=5"
        elif df.iloc[i, 3] > 5 and df.iloc[i, 3] <= 10:
            df.iloc[i, 3] = "5...10"
        elif df.iloc[i, 3] > 10:
            df.iloc[i, 3] = ">10"

    return df           


In [9]:
def acuracia(y_treino, y_teste):
    acertos = 0
    for i in range(len(y_teste)):
        if y_teste[i] == y_treino[i]: acertos += 1

    return acertos / len(y_treino)

In [10]:
categorizacao(df)
df.drop(["ocupacao_marido", "exposicao_midia"], axis=1, inplace=True)
df

,idade,escolaridade,escolaridade_marido,no_filhos,religiao,trabalha,indice_padrao_vida,contraceptivo
0,>39,1,3,5...10,1,1,4,1
1,>39,2,3,5...10,1,1,4,1
2,>39,3,2,5...10,1,1,3,1
3,32...39,3,3,5...10,1,1,2,1
4,<=26,4,4,<=5,1,1,3,1
...,...,...,...,...,...,...,...,...
1467,32...39,4,4,<=5,1,0,4,3
1468,32...39,4,4,<=5,1,1,4,3
1469,32...39,3,3,5...10,1,0,4,3
1470,32...39,3,3,<=5,1,0,2,3


# Classes

In [11]:
class Node():
    def __init__(self, coluna=None, limiar=None, esquerda=None, direita=None, *, valor=None):
        self.coluna = coluna
        self.limiar = limiar
        self.esquerda = esquerda
        self.direita = direita
        self.valor = valor

    def eh_folha(self):
        return self.valor is not None

In [31]:
class Arvore_Decisao():
    def __init__(self, metodos=3, altura_max=100, colunas=None):
        # PADRAO: EXEMPLO DE METODOS CONTRACEPTIVOS
        self.metodos = metodos
        self.altura_max = altura_max
        self.colunas = colunas
        self.raiz = None

    def arvore(self, df_treino, profundidade=0):
        y = df_treino.iloc[:, -1]
        x = df_treino.iloc[:, 0:-2]

        metodos = len(y.unique())
        linhas, colunas = x.shape

        # CONDICAO DE PARADA
        if self.altura_max <= profundidade or metodos==1 or linhas < self.metodos:
            # print(self.altura_max, profundidade)
            # print(metodos, linhas, self.metodos)
            valor_folha = self.valor_mais_comum(y)
            novo_no = Node(valor = valor_folha)
            return novo_no

        idx_colunas = random.sample(list(range(colunas))*2, self.colunas)
        melhor_coluna, melhor_limiar = self.divisao_melhor_ganho(x, y, idx_colunas)

        idx_esquerda = list(range(melhor_limiar)) if melhor_limiar != 0 else [0]
        idx_direita = list(range(melhor_limiar, len(x)))

        esquerda = self.arvore(df_treino.iloc[idx_esquerda, :], profundidade + 1)
        direita = self.arvore(df_treino.iloc[idx_direita, :], profundidade + 1)

        return Node(melhor_coluna, melhor_limiar, esquerda, direita)


    def valor_mais_comum(self, series):
        return series.value_counts().to_frame().iloc[0,0]

    def divisao_melhor_ganho(self, x, y, idx_colunas):
        melhor_ganho = -1
        aux = 0
        x.index = range(len(x))

        for i in idx_colunas:
            coluna_atual = x.iloc[:, i]
            nome_coluna = x.columns[i]
            limiares = len(coluna_atual)
            limiares = coluna_atual.unique()

            for item in limiares:
                ganho = calcular_entropia(pd.concat([x,y], axis=1), nome_coluna)

                if ganho > melhor_ganho:
                    melhor_ganho = ganho
                    divisao_coluna = i
                    divisao_limiar = x[x[nome_coluna]==item].index.tolist()[aux]
                    aux += 1

        return divisao_coluna, divisao_limiar

 
    def fit(self, df_treino):
        if not self.colunas:
            self.colunas = df_treino.shape[1]
        else: self.colunas = min(self.colunas, df_treino.shape[1])
        self.raiz = self.arvore(df_treino)


    def buscando_arvore(self, x, node):
        # print(node.coluna, node.valor, node.limiar)
        if node.eh_folha():
            return node.valor
        
        # print("Coluna:",node.coluna)
        # print("Limiar:",node.limiar)
        # print("x:",list(x.index))
        if list(x.index)[node.coluna] <= node.limiar:
            return self.buscando_arvore(x, node.esquerda)
        return self.buscando_arvore(x, node.direita)

    def previsao(self, x):
        resultado_previsao = []
        
        for coluna in range(x.shape[1]):
            df_aux = x.iloc[:, coluna]
            resultado_previsao.append(self.buscando_arvore(df_aux, self.raiz))

        return resultado_previsao


# Main()

In [13]:
df_treino, df_teste = treino_teste(df, 0.3)
x_treino = df_treino.iloc[:, 0:-2]
y_treino = df_treino.iloc[:, -1]

x_teste = df_teste.iloc[:, 0:-2]
y_teste = df_teste.iloc[:, -1]

In [32]:
dt = Arvore_Decisao(altura_max=40)
dt.fit(df_treino)

y_predicao = dt.previsao(x_teste)
print(y_predicao, y_teste)
valor_acuracia = acuracia(y_predicao, y_teste)
print("Valor acuracia:", valor_acuracia)

[7, 7, 7, 7, 7, 7] 0       1
4       1
5       1
6       1
7       1
       ..
1465    3
1466    3
1467    3
1470    3
1471    3
Name: contraceptivo, Length: 982, dtype: int64


KeyError: 1